# Zen Garden Design Analysis

## 1. Scrape

To collect our design data, we scrape csszengarden.com for design screenshots and associated styles. With over 200 designs, this should give us a good training set on how apply different styles and techniques.

In [ ]:
from data_collection.scraper import scrape_design
import asyncio

async def test_scraper(ids, batch_size=5):
    """
    Asynchronously scrape designs in batches to avoid overwhelming resources.
    
    Args:
        ids (list): List of design IDs to scrape
        batch_size (int): Number of designs to process concurrently
    """
    print(f"Starting scrape of {len(ids)} designs...")
    
    successful = 0
    failed = 0
    
    # Process in batches
    for i in range(0, len(ids), batch_size):
        batch = ids[i:i + batch_size]
        print(f"\nProcessing batch {i//batch_size + 1} ({len(batch)} designs)...")
        
        # Create tasks for current batch
        tasks = [scrape_design(design_id) for design_id in batch]
        
        # Run batch tasks concurrently
        results = await asyncio.gather(*tasks, return_exceptions=True)
        
        # Process batch results
        for design_id, result in zip(batch, results):
            if isinstance(result, Exception):
                print(f"Error scraping design {design_id}: {str(result)}")
                failed += 1
            else:
                print(f"Successfully scraped design {design_id}")
                successful += 1
        
        # Optional: Add delay between batches
        # await asyncio.sleep(1)
    
    print(f"\nScraping complete:")
    print(f"Successful: {successful}")
    print(f"Failed: {failed}")
    print(f"Total: {len(ids)}")

# Example usage with batch processing:
test_set = [f"{i:03d}" for i in range(1, 222)]
await test_scraper(test_set, batch_size=5)

Testing scraper with design 001...
001: Response status: 200
Success!
Testing scraper with design 002...
002: Response status: 200
Success!
Testing scraper with design 003...
003: Response status: 200


CancelledError: 

## 2. Analyze

Now, using the screenshots and styles we downloaded, we analyze the design for characteristics that will be useful for retrieval.

In [2]:
from data_collection.analyze_designs import analyze_screenshot
from pathlib import Path
import asyncio

async def test_analyzer():
    for design_id in test_set:
        design_path = Path(f"../designs/{design_id}")
        result = await analyze_screenshot(design_id, design_path)
        
        if result[1] is not None:  # If analysis was successful
            print(f"\nAnalysis for design {design_id}:")
            print(f"Description: {result[1]}")
            print(f"Categories: {', '.join(result[2])}")
            print(f"Visual Characteristics: {', '.join(result[3])}")

await test_analyzer()

Analyzing design 010...
Successfully analyzed design 010

Analysis for design 010:
Description: The design features a harmonious combination of earthy tones and soft gradients, creating a serene and inviting atmosphere. The layout leverages clear headings and subheadings for readability while integrating subtle decorative elements for visual interest. Functionally, the design is divided into a main content area and a sidebar, enhancing navigability.
Categories: Web Design, Educational, Inspirational, Aesthetic, Textual
Visual Characteristics: Earthy Color Palette, Soft Gradients, Informative Layout, Balanced Composition, Text-focused Design
